In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import load_model

from utils.custom_loss import mae_wrap_angle, mse_wrap_angle
from utils.load_data_raw import DataGenerator_raw, load_raw_all_h5
from utils.dataset_split import *
from utils.eval import *
from utils.utils import get_filelist, open_json

NUM_WORKER = 4

In [ ]:
# load filelist
model_dir = '../data/models_trained/'
filelist = get_filelist(model_dir)

# load data
dset_dir = '/media/feliximmohr/Storage/master_thesis/generated/database/raw/raw_nf10_mid/database_raw_nf10_scaledMM.h5'
feat, targ, ID_ref, pos_t, cond_t, par = load_raw_all_h5(dset_dir)

# load unscaled data
dset_nsc_dir = '/media/feliximmohr/Storage/master_thesis/generated/database/raw/raw_nf10_mid/database_raw_nf10.h5'
feat_nsc, targ_nsc, _, _, _, _ = load_raw_all_h5(dset_nsc_dir)

# create parameter dicts for the test batch generators
params = create_test_params(feat, targ, par, shuffle=False)
params_nsc = create_test_params(feat_nsc, targ_nsc, par, shuffle=False)

part = {}
for i in range(len(cond_t)):
    cond = cond_t.iloc[i]
    cond_ids_test, pos_ids_test, subject_ids_test = get_subset_ids([cond[0]], cond_t)
    part[cond[0]] = get_subset_sample_idx(ID_ref, cond_ids_test, pos_ids_test, subject_ids_test)

In [ ]:
h_flist = [x for x in filelist if 'history' in x]
pt_flist = [x for x in filelist if 'partition_test' in x]
m_flist = [x for x in filelist if x not in h_flist+pt_flist]

for i in range(len(m_flist)):
    s = ' -' if i<10 else '-'
    print(i, s, m_flist[i])
    print(i, s, h_flist[i])
    print(i, s, pt_flist[i])

In [ ]:
# load models, train history and test split IDs (partition['test'])

#wraptest
m64_mae = load_model(model_dir + m_flist[2], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h64_mae  = open_json(model_dir, h_flist[2])
pt64_mae = open_json(model_dir, pt_flist[2])

m64_maew = load_model(model_dir + m_flist[3], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h64_maew = open_json(model_dir, h_flist[3])
pt64_maew = open_json(model_dir, pt_flist[3])

m128_mae = load_model(model_dir + m_flist[5], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128_mae  = open_json(model_dir, h_flist[5])
pt128_mae = open_json(model_dir, pt_flist[5])

m128_maew = load_model(model_dir + m_flist[10], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128_maew = open_json(model_dir, h_flist[10])
pt128_maew = open_json(model_dir, pt_flist[10])

m256_mae = load_model(model_dir + m_flist[14], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h256_mae  = open_json(model_dir, h_flist[14])
pt256_mae = open_json(model_dir, pt_flist[14])

m256_maew = load_model(model_dir + m_flist[15], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h256_maew = open_json(model_dir, h_flist[15])
pt256_maew = open_json(model_dir, pt_flist[15])                      

#normtest
m128_maew_nn = load_model(model_dir + m_flist[9], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128_maew_nn = open_json(model_dir, h_flist[9])
pt128_maew_nn = open_json(model_dir, pt_flist[9])

#toptest
m128_tt = load_model(model_dir + m_flist[6], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128_tt = open_json(model_dir, h_flist[6])
pt128_tt = open_json(model_dir, pt_flist[6])

m128bn_bs2048_tt = load_model(model_dir + m_flist[13], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128bn_bs2048_tt = open_json(model_dir, h_flist[13])
pt128bn_bs2048_tt = open_json(model_dir, pt_flist[13])

m128bn_bs256_tt = load_model(model_dir + m_flist[11], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128bn_bs256_tt = open_json(model_dir, h_flist[11])
pt128bn_bs256_tt = open_json(model_dir, pt_flist[11])

m128_128_64_tt = load_model(model_dir + m_flist[4], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128_128_64_tt = open_json(model_dir, h_flist[4])
pt128_128_64_tt = open_json(model_dir, pt_flist[4])

m512_256_tt = load_model(model_dir + m_flist[16], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h512_256_tt = open_json(model_dir, h_flist[16])
pt512_256_tt = open_json(model_dir, pt_flist[16])

m32_16_tt = load_model(model_dir + m_flist[0], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h32_16_tt  = open_json(model_dir, h_flist[0])
pt32_16_tt = open_json(model_dir, pt_flist[0])

## Test: normalized vs. not normalized

    Model 1: 128-128-1    m128_maew_wraptest   vs.  m128_maew_normtest_neg


In [ ]:
# Model 1 normalized: m128_maew
tbg128_maew = model_complete_eval(m128_maew, h128_maew, pt128_maew, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_maew, tbg128_maew, b_idx=0)

In [ ]:
# Model 1 not normalized: m128_maew_nn
tbg128_maew_nn = model_complete_eval(m128_maew_nn, h128_maew_nn, pt128_maew_nn, params_nsc, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_maew_nn, tbg128_maew_nn, b_idx=0)

## Test: custom vs. non-custom loss function

    Model 1: 64-32-1      m64_maew_wraptest   vs.  m64_mae_wraptest
    Model 2: 128-128-1    m128_maew_wraptest  vs.  m128_mae_wraptest
    Model 3: 256-128-1    m256_maew_wraptest  vs.  m256_mae_wraptest


In [ ]:
# Model 1 custom loss: m64_maew
tbg64_maew = model_complete_eval(m64_maew, h64_maew, pt64_maew, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m64_maew, tbg64_maew, b_idx=0)

In [ ]:
# Model 1 mae loss: m64_mae
tbg64_mae = model_complete_eval(m64_mae, h64_mae, pt64_mae, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m64_mae, tbg64_mae, b_idx=0)

In [ ]:
# Model 2 custom loss: m128_maew
tbg128_maew = model_complete_eval(m128_maew, h128_maew, pt128_maew, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_maew, tbg128_maew, b_idx=0)

In [ ]:
# Model 2 mae loss: m128_mae
tbg128_mae = model_complete_eval(m128_mae, h128_mae, pt128_mae, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_mae, tbg128_mae, b_idx=0)

In [ ]:
# Model 3 custom loss: m256_maew
tbg256_maew = model_complete_eval(m256_maew, h256_maew, pt256_maew, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m256_maew, tbg256_maew, b_idx=0)

In [ ]:
# Model 3 mae loss: m3_mae
tbg256_mae = model_complete_eval(m256_mae, h256_mae, pt256_mae, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m256_mae, tbg256_mae, b_idx=0)

## Test: Model/Net Topology
    Model 1:         128-128-1
    Model 1 + BN :   128-BN-128-BN-1
    Model 2:         128-128-64-1
    Model 3:         512-256-1
    Model 4:         32-16-1

In [ ]:
# Model 1: m128_tt
tbg128_tt = model_complete_eval(m128_tt, h128_tt, pt128_tt, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_tt, tbg128_tt, b_idx=0)

In [ ]:
# Model 1 + BN: m128bn_bs2048_tt
tbg128bn_bs2048_tt = model_complete_eval(m128bn_bs2048_tt, h128bn_bs2048_tt, pt128bn_bs2048_tt, params, batch_size=2048, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128bn_bs2048_tt, tbg128bn_bs2048_tt, b_idx=0)

In [ ]:
# Model 2: m128_128_64_tt
tbg128_128_64_tt = model_complete_eval(m128_128_64_tt, h128_128_64_tt, pt128_128_64_tt, params_nsc, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_128_64_tt, tbg128_128_64_tt, b_idx=0)

In [ ]:
# Model 3: m512_256_tt
tbg512_256_tt = model_complete_eval(m512_256_tt, h512_256_tt, pt512_256_tt, params_nsc, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m512_256_tt, tbg512_256_tt, b_idx=0)

In [ ]:
# Model 4: m32_16_tt not normalized?
tbg32_16_tt = model_complete_eval(m32_16_tt, h32_16_tt, pt32_16_tt, params_nsc, batch_size=1024, workers=NUM_WORKER)

#predict on model
pred, y = model_pred_on_gen_batch(m32_16_tt, tbg32_16_tt, b_idx=0)

### Results

In [ ]:
# Model 1: m128_tt
mae_p, mse_p, loc_p = model_eval_pos(m128_tt, h128_tt, part['NFCHOA_R006'], params, ID_ref, batch_size=1000, workers=NUM_WORKER)